In [10]:
from config import *

import gensim

import glob
import os
import re

import numpy as np
import pandas as pd

import json

In [11]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim.models import ldamodel

In [12]:
THEME = 'Inflation'
theme = 'Inflation'

In [13]:
# Load dictionary and corpus
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (THEME, THEME))

2021-04-26 00:26:35,433 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict
2021-04-26 00:26:35,437 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict', 'datetime': '2021-04-26T00:26:35.437710', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}


In [14]:
# 1. Look for ids of related words to theme in the dictionary

In [15]:
words = pd.read_csv(TEMP_PATH + '/%s_words.csv' % THEME.lower(), index_col=0)

In [16]:
year_name_list = dict()
year_id_list = dict()
for col in words.columns:
    reqd_words = list(words[col].values) + [THEME.lower()]
    reqd_ids = [k for k,v in dictionary_all.items() if v in reqd_words]
    year_name_list[col] = [dictionary_all[i] for i in reqd_ids]
    year_id_list[col] = reqd_ids

In [17]:
year_name_list

{'1996': ['inflation',
  'gdp',
  'unemployment',
  'pressures',
  'wage',
  'cpi',
  'economy'],
 '1997': ['inflation', 'unemployment', 'pressures', 'wage', 'cpi', 'economy'],
 '1998': ['inflation', 'gdp', 'unemployment', 'wage', 'cpi', 'economy'],
 '1999': ['inflation', 'gdp', 'wage', 'cpi', 'economy'],
 '2000': ['inflation', 'gdp', 'pressures', 'wage', 'cpi', 'economy'],
 '2001': ['inflation',
  'gdp',
  'unemployment',
  'cpi',
  'ecb',
  'growth',
  'economy'],
 '2002': ['inflation', 'gdp', 'cpi', 'ecb', 'economy'],
 '2003': ['inflation',
  'fed',
  'gdp',
  'payrolls',
  'cpi',
  'policymakers',
  'ecb',
  'economy'],
 '2004': ['inflation',
  'fed',
  'gdp',
  'payrolls',
  'cpi',
  'greenspan',
  'bernanke'],
 '2005': ['easing', 'inflation', 'fed', 'gdp', 'payrolls', 'cpi', 'greenspan'],
 '2006': ['inflation', 'fed', 'gdp', 'payrolls', 'cpi', 'bernanke'],
 '2007': ['easing',
  'inflation',
  'fed',
  'gdp',
  'recession',
  'cpi',
  'policymakers',
  'tightening'],
 '2008': ['ea

In [18]:
years = [str(year) for year in range(START_YEAR, END_YEAR + 1)]

In [19]:
good_words_count_per_year = dict()
for year in years:
    good_words_count_per_year[year] = []
    with open(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year)) as f:
        a = json.load(f)
        for article in a:
            good_words_count_per_year[year].append(len([word for word in article if word in year_name_list[year]]))
    print(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year), 'done!')

/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1996.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1997.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1998.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1999.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2000.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2001.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2002.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2003.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2004.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2005.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2006.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2007.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2008.json done!

In [20]:
yearly_slices = []
for year in years:
    # Get the 1% threshold count (the top 1% of most relevant articles)
    x = good_words_count_per_year[year]
    x.sort()
    threshold = x[-int(len(x)*0.01):][0]
    year_article_count = 0
    with open(TOKENIZED_ARTICLES_PATH % (theme, theme, year)) as f:
        article_index = 0
        all_articles = json.load(f)
        for articles in all_articles:
            if good_words_count_per_year[year][article_index] >= threshold:
                year_article_count += 1    
            article_index += 1
    yearly_slices.append(year_article_count)
    print(TOKENIZED_ARTICLES_PATH % (theme, theme, year), 'done!')
    print('Count for the year:', year_article_count, 'Vs original:', len(all_articles))

# dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (theme, theme))

/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1996.json done!
Count for the year: 335 Vs original: 30955
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1997.json done!
Count for the year: 480 Vs original: 38966
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1998.json done!
Count for the year: 420 Vs original: 40768
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1999.json done!
Count for the year: 774 Vs original: 63481
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2000.json done!
Count for the year: 718 Vs original: 60976
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2001.json done!
Count for the year: 924 Vs original: 74314
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2002.json done!
Count for the year: 504 Vs original: 39424
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2003.json done!
Count for the year: 1391 Vs original: 132201
/work/ms5941/NLP/Temp/Inflation/Inflat

In [21]:
sum(yearly_slices)

29940

In [22]:
with open(TEMP_PATH + '/%s/%s_yearly_slices.txt' % (THEME, THEME), 'w') as f:
    json.dump(yearly_slices, f)

In [ ]:
# with open(TEMP_PATH + '/%s/%s_yearly_slices.txt' % (THEME, THEME), 'r') as f:
#     yearly_slices = json.load(f)

In [23]:
print(yearly_slices)

[335, 480, 420, 774, 718, 924, 504, 1391, 887, 1457, 1371, 1868, 1489, 1235, 2190, 1434, 1734, 1205, 717, 1291, 1283, 758, 2074, 2583, 818]


In [24]:
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (THEME, THEME))
corpus_all = gensim.corpora.MmCorpus(TEMP_PATH + '/%s/%s_less_restricted.mm' % (THEME, THEME))


2021-04-26 00:41:41,123 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict
2021-04-26 00:41:41,125 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict', 'datetime': '2021-04-26T00:41:41.125477', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}
2021-04-26 00:41:41,147 : INFO : loaded corpus index from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.mm.index
2021-04-26 00:41:41,149 : INFO : initializing cython corpus reader from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.mm
2021-04-26 00:41:41,178 : INFO : accepted corpus with 29940 documents, 1623 features, 3903782 non-zero entries


In [25]:
import numpy as np

cum_yearly_slices = np.cumsum(yearly_slices)

corpus_by_year = dict()
corpus_by_year[START_YEAR] = corpus_all[:cum_yearly_slices[0]]
for i in range(1, 25):
    corpus_by_year[START_YEAR + i] = corpus_all[cum_yearly_slices[i-1]:cum_yearly_slices[i]]

In [26]:
len(corpus_by_year[2009]) 

1235

In [38]:
import matplotlib.pyplot as plt

def get_avg_topic_probabilities(lda, corp, num_topics):
    """
    For the given LDA model and corpus, get the aggregate probability of each topic 
    (by iterating over each document in the corpus, adding up individual probabilities and aggregating)
    Then, divide by the total number of documents in the corpus to get the average 
    topic probabilities for the corpus.
    
    """
    all_topics_probabilities = np.zeros(num_topics)
    for article in corp:
        article_topics = lda.get_document_topics(article)
        topic_vec = np.zeros(num_topics)
        for k, prob in article_topics:
            topic_vec[k] = prob
        all_topics_probabilities += topic_vec
    
    # Avg topic probabilities
    avg_topic_probabilities = all_topics_probabilities/float(len(corp))
    
    return avg_topic_probabilities


def get_top_ten_topics_for_year(year, lda, avg_topic_probabilities):
    """
    Using the average topic probabilites, rank the topics and 
    return the top ten topics for a year.
    
    """
    # Get top 10 topics for each year
    indices = (-avg_topic_probabilities).argsort()[:10]

    top_topics_words = dict()
    top_topics_words[year] = dict()
    rank = 1
    for ind in indices:
        top_words = lda.show_topic(ind, topn=10)
        words, probs = zip(*top_words)
        top_topics_words[year][rank] = top_words
        rank += 1

    df = pd.DataFrame.from_dict({(i,j): [x[0] for x in top_topics_words[i][j]] for i in top_topics_words.keys() 
                            for j in top_topics_words[i].keys()}).T
    return df, top_topics_words



def get_largest_popularity_shifts(avg_topics_all, num_topics=50, passes=50, iterations=400, 
                                  eval_every=None, update_every=0, alpha='auto', eta='auto'):

    # Get topic ranks for each year
    yearly_ranks = dict()

    temp = (-avg_topics_all[0]).argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(avg_topics_all[0]))
    yearly_ranks[2019] = ranks

    temp = (-avg_topics_all[1]).argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(avg_topics_all[1]))
    yearly_ranks[2020] = ranks
    
    shift_in_popularity = yearly_ranks[2019] - yearly_ranks[2020]
    
    top_shifts = dict()
    # Top gains indices
    top_shifts['upward'] = (shift_in_popularity).argsort()[:5]
    
    # Top drops indices
    top_shifts['downward'] = (-shift_in_popularity).argsort()[:5]

    for trend in ['upward', 'downward']:
        print('Top %s shifts in popularity (2019 to 2020)' % trend)
        top_shift_topics_words = dict()
        rank = 1
        for ind in top_shifts[trend]:
            tempfile = datapath('model_'+ '_'.join([str(num_topics), str(passes), str(iterations), str(alpha), str(eta)]))
            if os.path.exists(tempfile):
                lda = gensim.models.LdaModel.load(tempfile)    
            top_words = lda.show_topic(ind, topn=12)
            words, probs = zip(*top_words)
            top_shift_topics_words[rank] = top_words
            rank += 1

        display(pd.DataFrame.from_dict({x:[y[0] for y in top_shift_topics_words[x]] for x in top_shift_topics_words}).T)
    

In [36]:
def generate_lda_model(corpus, dictionary, num_topics=10, passes=25, 
                       iterations=400, eval_every=None, update_every=0, 
                       alpha='auto', eta='auto'):

    lda = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, alpha='auto', eta='auto',
                                 iterations=iterations, num_topics=num_topics, passes=passes, 
                                 eval_every=eval_every, update_every = update_every)
    
    # Save lda model
    tempfile = TEMP_PATH + '/%s/%s_LDA_model_' % (THEME, THEME) + '_'.join([str(num_topics), str(passes), str(iterations), str(alpha), str(eta)]) 
    lda.save(tempfile)
    
    return lda



def get_topics(corpus_all, dictionary_all, corpus_by_year, num_topics=10, passes=25, iterations=400, 
               eval_every=None, update_every=0, alpha='auto', eta='auto'):
    """
    Get the top topics for each year, based on an LDA model created using documents of both years
    
    """
    # Check if a model with the same config already exists. 
    # If it does, load the model instead of generating a new one
    tempfile = TEMP_PATH + '/%s/%s_LDA_model_' % (THEME, THEME) + '_'.join([str(num_topics), str(passes), str(iterations), str(alpha), str(eta)]) 
    if os.path.exists(tempfile):
        lda = gensim.models.LdaModel.load(tempfile)
    else:
        lda = generate_lda_model(corpus_all, dictionary_all, num_topics, passes, 
                                 iterations, eval_every, update_every, alpha, eta)

    avg_topics_all = []
    for year in range(START_YEAR, END_YEAR + 1):
        avg_topic_probabilities = get_avg_topic_probabilities(lda, corpus_by_year[year], num_topics)
        df, top_topic_words = get_top_ten_topics_for_year(year, lda, avg_topic_probabilities)
        display(df)
        avg_topics_all.append(avg_topic_probabilities)

    # Plot avg topic distribution graphs
    fig, ax = plt.subplots(1, 2, figsize=(15,5))

    ax[0].bar(range(len(avg_topics_all[0])), avg_topics_all[0]*100)
    ax[1].bar(range(len(avg_topics_all[1])), avg_topics_all[1]*100)
    plt.setp(ax, ylim=(0, 20))
    plt.show()

    return avg_topics_all

In [ ]:
avg_topics_all = get_topics(corpus_all, dictionary_all, corpus_by_year, num_topics=15)

2021-04-26 02:03:35,852 : INFO : using autotuned alpha, starting with [0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667]
2021-04-26 02:03:35,854 : INFO : using serial LDA version on this node
2021-04-26 02:03:35,857 : INFO : running batch LDA training, 15 topics, 25 passes over the supplied corpus of 29940 documents, updating model once every 29940 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2021-04-26 02:03:36,020 : INFO : PROGRESS: pass 0, at document #2000/29940
2021-04-26 02:03:48,942 : INFO : optimized alpha [0.052263323, 0.044922367, 0.048453897, 0.058766693, 0.04996645, 0.053361673, 0.04557566, 0.047570415, 0.047592096, 0.05980467, 0.048661433, 0.048281375, 0.049460262, 0.047281563, 0.050988495]
2021-04-26 02:03:49,154 : INFO : PROGRESS: pass 0, at document #4000/29940
2021-04-26 02:04:04

2021-04-26 02:08:36,475 : INFO : PROGRESS: pass 1, at document #14000/29940
2021-04-26 02:08:44,222 : INFO : optimized alpha [0.0118677635, 0.008828902, 0.014805695, 0.04212148, 0.0127675, 0.018763013, 0.010828909, 0.013137182, 0.024449682, 0.01853006, 0.010797266, 0.0128673585, 0.015761895, 0.010387859, 0.012479767]
2021-04-26 02:08:44,468 : INFO : PROGRESS: pass 1, at document #16000/29940
2021-04-26 02:08:50,942 : INFO : optimized alpha [0.011689831, 0.009548051, 0.014585014, 0.038009547, 0.012571252, 0.018453281, 0.010951991, 0.01300552, 0.027156606, 0.01816489, 0.010600126, 0.012534592, 0.015237059, 0.010408085, 0.01232541]
2021-04-26 02:08:51,226 : INFO : PROGRESS: pass 1, at document #18000/29940
2021-04-26 02:08:58,601 : INFO : optimized alpha [0.011596731, 0.00938574, 0.014654731, 0.03798643, 0.012675713, 0.018000772, 0.011186326, 0.012964474, 0.028961727, 0.017888185, 0.010458774, 0.012262683, 0.014742553, 0.010274606, 0.012438212]
2021-04-26 02:08:58,899 : INFO : PROGRESS: p

2021-04-26 02:11:01,704 : INFO : optimized alpha [0.010448678, 0.007331315, 0.015819628, 0.023051213, 0.014271341, 0.013323141, 0.014704699, 0.012482776, 0.034941748, 0.013876545, 0.008639371, 0.008796165, 0.009057965, 0.009160625, 0.011866662]
2021-04-26 02:11:02,015 : INFO : PROGRESS: pass 2, at document #29940/29940
2021-04-26 02:11:06,618 : INFO : optimized alpha [0.010276774, 0.0072023286, 0.015455704, 0.023383535, 0.014039294, 0.012901764, 0.014764835, 0.012272251, 0.036477778, 0.013481815, 0.0085466225, 0.008730208, 0.008872802, 0.0089920815, 0.01166738]
2021-04-26 02:11:06,660 : INFO : topic #1 (0.007): 0.010*"futures" + 0.008*"wealth" + 0.006*"options" + 0.006*"dow" + 0.006*"management" + 0.006*"ftse" + 0.005*"help" + 0.005*"indicators" + 0.005*"view" + 0.005*"group"
2021-04-26 02:11:06,662 : INFO : topic #10 (0.009): 0.007*"washington" + 0.006*"claims" + 0.006*"mln" + 0.006*"time" + 0.005*"meeting" + 0.005*"sales" + 0.004*"president" + 0.004*"consumer" + 0.004*"annual" + 0.00

2021-04-26 02:12:21,338 : INFO : PROGRESS: pass 4, at document #6000/29940
2021-04-26 02:12:24,047 : INFO : optimized alpha [0.010185611, 0.006370862, 0.01680307, 0.01490171, 0.017710794, 0.012425433, 0.016540673, 0.013292846, 0.02827922, 0.012862063, 0.0076238643, 0.007088862, 0.0065787565, 0.009183893, 0.011534441]
2021-04-26 02:12:24,233 : INFO : PROGRESS: pass 4, at document #8000/29940
2021-04-26 02:12:27,655 : INFO : optimized alpha [0.010149915, 0.006284854, 0.016805127, 0.014398482, 0.018075787, 0.012631688, 0.01649091, 0.013569693, 0.026402999, 0.012822179, 0.0076323687, 0.007027602, 0.00648754, 0.009394868, 0.011732381]
2021-04-26 02:12:27,831 : INFO : PROGRESS: pass 4, at document #10000/29940
2021-04-26 02:12:31,316 : INFO : optimized alpha [0.010308765, 0.006208207, 0.017016977, 0.013927005, 0.018701645, 0.012682003, 0.01637483, 0.013949586, 0.024776481, 0.012744779, 0.007600761, 0.006960422, 0.0064084264, 0.009588499, 0.0119476495]
2021-04-26 02:12:31,526 : INFO : PROGRES

2021-04-26 02:13:47,097 : INFO : PROGRESS: pass 5, at document #20000/29940
2021-04-26 02:13:50,842 : INFO : optimized alpha [0.009975827, 0.0074211475, 0.016973434, 0.0107812965, 0.019645555, 0.0121540325, 0.017549964, 0.014477934, 0.024395367, 0.011828801, 0.006928023, 0.0060015186, 0.005032325, 0.009816029, 0.010880303]
2021-04-26 02:13:51,130 : INFO : PROGRESS: pass 5, at document #22000/29940
2021-04-26 02:13:54,487 : INFO : optimized alpha [0.009940851, 0.0073049697, 0.01685179, 0.011072077, 0.01953753, 0.011839184, 0.017762959, 0.014298563, 0.02470316, 0.011675351, 0.0068907267, 0.0059667774, 0.004979133, 0.009697561, 0.01075228]
